In [ ]:
# 캘리 파일 테스트

In [ ]:
import numpy as np
import cv2
import os

In [ ]:
CALIB_FILE = "stereo_calib.npz"
CAMERA_INDEX = 0

# 해상도
FRAME_WIDTH = 1280 
FRAME_HEIGHT = 480

In [ ]:
# 캘리 데이터
if not os.path.exists(CALIB_FILE):
    print("에러: 'stereo_calib.npz' 파일이 없습니다.")
    exit()

print("캘리 데이터 로드")
data = np.load(CALIB_FILE)
mtxL, distL = data['mtxL'], data['distL']
mtxR, distR = data['mtxR'], data['distR']
R, T = data['R'], data['T']

# 카메라
cap = cv2.VideoCapture(CAMERA_INDEX)
cap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc(*'MJPG'))
cap.set(cv2.CAP_PROP_FRAME_WIDTH, FRAME_WIDTH)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, FRAME_HEIGHT)

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
half_width = width // 2

if width == 0:
    print("카메라 연결 실패")
    exit()

print(width)
print(height)

In [ ]:
# Rectification 변환 행렬 계산 (한 번만 수행)
# alpha=0: 검은 테두리 제거 (확대됨), alpha=1: 모든 픽셀 포함 (검은 테두리 보임)
# 기본적으로 -1이나 0을 많이 씁니다.
R1, R2, P1, P2, Q, roi1, roi2 = cv2.stereoRectify(
    mtxL, distL, mtxR, distR, (half_width, height), R, T, alpha=0
)

# 매핑 함수 생성 (속도 최적화)
map1_L, map2_L = cv2.initUndistortRectifyMap(mtxL, distL, R1, P1, (half_width, height), cv2.CV_16SC2)
map1_R, map2_R = cv2.initUndistortRectifyMap(mtxR, distR, R2, P2, (half_width, height), cv2.CV_16SC2)

print(f"{width}x{height}")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 좌우 분할
    imgL_raw = frame[:, :half_width]
    imgR_raw = frame[:, half_width:]

    # 왜곡 보정 및 정렬 (Remap)
    imgL_rect = cv2.remap(imgL_raw, map1_L, map2_L, cv2.INTER_LINEAR)
    imgR_rect = cv2.remap(imgR_raw, map1_R, map2_R, cv2.INTER_LINEAR)

    # 비교를 위해 원본과 보정본을 각각 하나로 합침
    # 위쪽: 원본 (Raw), 아래쪽: 보정본 (Rectified)
    res_raw = np.hstack((imgL_raw, imgR_raw))
    res_rect = np.hstack((imgL_rect, imgR_rect))

    cv2.imshow("", frame)
    
    # 보정된 영상에 가로선 그리기 (정렬 확인용)
    # 30픽셀 간격으로 선을 그립니다.
    for line_y in range(0, res_rect.shape[0], 30):
        # 원본에는 빨간선 (비교용)
        cv2.line(res_raw, (0, line_y), (res_raw.shape[1], line_y), (0, 0, 255), 1)